-----

## Transforming Word Similarity / Association datasets

In [1]:
%matplotlib inline
import pickle
from os import listdir, makedirs
from os.path import join, isfile, exists, dirname
import gc
import re
import math

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
%pylab inline
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import TfidfModel, LdaModel
from gensim.models import Word2Vec, Doc2Vec, FastText
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from tqdm._tqdm_notebook import tqdm_notebook

from constants import *
from utils import load, init_logging, tprint, TopicsLoader
from topic_reranking import Reranker

#from eval_lda import eval_coherence

tqdm_notebook.pandas()
pd.options.display.max_columns = 30
pd.options.display.max_rows = 2000

Populating the interactive namespace from numpy and matplotlib


In [165]:
# Gurevych datasets

names = ['Word1', 'Word2', 'Gold', 'Pos1', 'Pos2']
def gur(size):
    return (
        pd
        .read_csv(join(DATA_BASE, 'gurevych_datasets', f'wortpaare{size}.gold.pos.txt'), sep=':', names=names, comment='#')
        .assign(Dataset=f'gur{size}')
        .set_index('Dataset', append=True)
    )

df = pd.concat([gur(350), gur(222), gur(65)])
df.to_csv(join(ETL_PATH, 'gurevych_datasets.csv'))
df

,,Word1,Word2,Gold,Pos1,Pos2
,Dataset,,,,,
0,gur350,Absage,ablehnen,3.500,n,v
1,gur350,Absage,Stellenanzeige,1.880,n,n
2,gur350,Affe,Gepäckkontrolle,0.130,n,n
3,gur350,Affe,Makake,4.000,n,n
4,gur350,Afrika,historisch,1.000,n,a
5,gur350,Agentur,Irrtum,0.000,n,n
6,gur350,Airbag,Kopfairbag,3.880,n,n
7,gur350,analysieren,Analyse,3.880,v,n
8,gur350,Ansehen,Schaden,0.880,n,n


In [14]:
# SimLex999 German Dataset

file_en = join(DATA_BASE, 'SimLex-999', 'SimLex-999.txt')
df_en = pd.read_csv(file_en, sep='\t').assign(dataset=f'SimLex999_en')
file_de = join(DATA_BASE, 'SimLex_ALL_Langs_TXT_Format', 'MSimLex999_German.txt')
df = pd.read_csv(file_de, sep=',').assign(dataset=f'SimLex999_de')
df['POS'] = df_en.POS
df['avg'] = df.loc[:, '1':'13'].mean(axis=1)
df['sd'] = df.loc[:, '1':'13'].std(axis=1)
df = df.reset_index().rename(columns={'index': 'topic_idx'}).set_index(['dataset', 'topic_idx'])

#df.to_csv(join(ETL_PATH, 'simlex999.csv'))
df

Word1                Word2   1   2   3  \
dataset      topic_idx                                                          
SimLex999_de 0                           alt                  neu   0   0   0   
             1                          klug          intelligent  10   9   9   
             2                        schwer            schwierig   9   9  10   
             3                     glücklich             fröhlich  10   8  10   
             4                        schwer               leicht   0   0   1   
             5                       schnell               rasant  10   9   9   
             6                     glücklich              freudig  10   8  10   
             7                          kurz                 lang   0   0   1   
             8                          dumm                 blöd  10   9   9   
             9                     sonderbar         eigentümlich   8   9   9   
             10                         weit                  eng   0   0   0   
             11                     schlecht          schrecklich  10   0   9   
             12                       leicht            schwierig   0   0   0   
             13                     schlecht            furchtbar  10   0   9   
             14                       schwer              einfach   0   0   0   
             15                         klug                 blöd   0   0   0   
             16                   wahnsinnig             verrückt  10   9   9   
             17                    glücklich                 irre   0   0   1   
             18                         groß               riesig  10   9   9   
             19                         hart                  zäh   7   9   5   
             20                          neu               frisch  10   9   8   
             21                       scharf               stumpf   0   0   0   
             22                        rasch               rasant  10   9   9   
             23                         blöd              töricht  10   8   5   
             24                    wunderbar          fantastisch  10   9  10   
             25                 eigentümlich              seltsam   0   9   9   
             26                    glücklich            ärgerlich   0   0   1   
             27                          eng                breit   0   0   1   
             28                      einfach               leicht  10   9   9   
             29                          alt               frisch   0   0   0   
             30                 offentkundig       offensichtlich   6   9   5   
             31                    preiswert               billig  10   9   5   
             32                         nett            großmütig  10   9   9   
             33                    sonderbar               normal   0   0   1   
             34                    sonderbar              seltsam   8   9   9   
             35                     schlecht          unmoralisch  10   8   5   
             36                      traurig               lustig   0   0   0   
             37                    wunderbar            großartig  10   9  10   
             38                     schuldig             beschämt   7   0   1   
             39                        schön            wunderbar   9   9   9   
             40               zuversichtlich               sicher  10   9   5   
             41                         blöd       unterbelichtet  10   9   9   
             42                  umfangreich                 groß  10   9   5   
             43                         nett              grausam   0   0   1   
             44                   ungeduldig               nervös   7   9   9   
             45                         groß                breit  10   0   1   
             46                        stark                stolz   0   0   7   
             47                      unnötig            notwendig   0 

In [41]:
# WS353 German Dataset

file = join(DATA_BASE, 'WS353_All_Langs_TXT_Format', 'MWS353_German.txt')
f_sim = join(DATA_BASE, 'WS353_ALL_Langs_SIM_TXT_Format', 'WS353-german-sim.txt')
f_rel = join(DATA_BASE, 'WS353_ALL_Langs_REL_TXT_Format', 'WS353-german-rel.txt')

df = pd.read_csv(file, sep=',').assign(dataset=f'WS353_de')
df_sim = pd.read_csv(f_sim, sep=',').assign(dataset=f'WS353_sim')
df_rel = pd.read_csv(f_rel, sep=',').assign(dataset=f'WS353_rel')

df = df.join(df_sim.set_index(['Word1', 'Word2', 'Average_Score']), on=['Word1', 'Word2', 'Average Score'], how='outer', rsuffix='_sim')
df = df.join(df_rel.set_index(['Word1', 'Word2', 'Average_Score']), on=['Word1', 'Word2', 'Average Score'], how='outer', rsuffix='_rel')
df[['dataset_sim', 'dataset_rel']] = df[['dataset_sim', 'dataset_rel']].applymap(lambda x: '' if isinstance(x, float) else x)

df['avg'] = df.loc[:, '1':'13'].mean(axis=1)
df['sd'] = df.loc[:, '1':'13'].std(axis=1)
df = df.reset_index().rename(columns={'index': 'topic_idx'}).set_index(['dataset', 'topic_idx'])

#df.to_csv(join(ETL_PATH, 'ws353.csv'))
df

Word1             Word2   1   2   3   4   5  \
dataset  topic_idx                                                            
WS353_de 0                      Liebe               Sex   8   9   8   9   9   
         1                      Tiger             Katze   8   7   7   8   7   
         2                      Tiger             Tiger  10  10  10  10  10   
         3                       Buch            Papier   9   8   8   4   8   
         4                   Computer          Tastatur   8   7   8   7   9   
         5                   Computer          Internet   8   7   9   7   7   
         6                   Flugzeug              Auto   7   0   7   0   6   
         7                        Zug              Auto   7   0   7   0   5   
         8                    Telefon     Kommunikation  10   8   8   6   8   
         9                  Fernseher             Radio   8   0   8   0   8   
         10                    Medien             Radio   6   8   8   9   4   
         11                    Drogen         Mißbrauch   5   8   5   8   9   
         12                      Brot            Butter   6   0   7   3   8   
         13                     Gurke         Kartoffel   8   0   8   0   2   
         14                      Arzt  Krankenschwester   8   2   7   1   8   
         15                 Professor            Doktor   6   7   8   5   7   
         16                   Student         Professor   7   2   8   0   5   
         17                      klug           Student   8   2   5   2   6   
         18                      klug              dumm   8   0   5   0   8   
         19               Unternehmen             Aktie   7   7   7   5   8   
         20                     Aktie             Börse   9   9   8   8  10   
         21                    Vorrat           Telefon   0   0   0   0   0   
         22                    Vorrat                CD   0   0   2   0   0   
         23                    Vorrat            Jaguar   0   0   0   0   0   
         24                    Vorrat                Ei   3   0   6   0   1   
         25             Fruchtbarkeit                Ei   7   9   7   8   8   
         26                     Aktie              Live   0   2   0   1   5   
         27                     Aktie             Leben   0   0   0   0   5   
         28                      Buch        Bibliothek   8   7   8   4   9   
         29                      Bank              Geld   8   9   9   5   8   
         30                      Holz              Wald   9   7   8   7   7   
         31                      Geld           Bargeld  10  10   9  10  10   
         32                 Professor             Gurke   0   0   0   0   0   
         33                     König              Kohl   0   0   0   0   0   
         34                     König           Königin  10  10  10  10  10   
         35                     König              Turm   2   2   7   6   8   
         36                  Bischoff             Rabbi   9   5   7   0   2   
         37                 Jerusalem            Israel   9   8   8   8  10   
         38                 Jerusalem   Palestinensisch   7   8   8   8   5   
         39                    Heilig               Sex   0   0   0   3   0   
         40                    Ficken               Sex   8   9   7   9   9   
         41                  Maradona           Fußball   8   8   8   7   7   
         42                   Fußball        Basketball   8   0   8   0   6   
         43                   Fußball            Tennis   7   0   6   0   6   
         44                    Tennis          Schläger   8   7   6   6   6   
         45                    Arafat           Frieden   5   2   0   2   3   
         46                    Arafat            Terror   5   0   0   0   6   
         47                    Arafat           Jackson   2   0   0   0   0   
         48                    Gesetz            Anwalt   8   8   9   7   8   
   

In [44]:
file = join(ETL_PATH, 'simlex999.csv')
df = pd.read_csv(file, header=0, index_col=[0, 1])
df[['Word1', 'Word2']]

Word1                Word2
dataset      topic_idx                                           
SimLex999_de 0                           alt                  neu
             1                          klug          intelligent
             2                        schwer            schwierig
             3                     glücklich             fröhlich
             4                        schwer               leicht
             5                       schnell               rasant
             6                     glücklich              freudig
             7                          kurz                 lang
             8                          dumm                 blöd
             9                     sonderbar         eigentümlich
             10                         weit                  eng
             11                     schlecht          schrecklich
             12                       leicht            schwierig
             13                     schlecht            furchtbar
             14                       schwer              einfach
             15                         klug                 blöd
             16                   wahnsinnig             verrückt
             17                    glücklich                 irre
             18                         groß               riesig
             19                         hart                  zäh
             20                          neu               frisch
             21                       scharf               stumpf
             22                        rasch               rasant
             23                         blöd              töricht
             24                    wunderbar          fantastisch
             25                 eigentümlich              seltsam
             26                    glücklich            ärgerlich
             27                          eng                breit
             28                      einfach               leicht
             29                          alt               frisch
             30                 offentkundig       offensichtlich
             31                    preiswert               billig
             32                         nett            großmütig
             33                    sonderbar               normal
             34                    sonderbar              seltsam
             35                     schlecht          unmoralisch
             36                      traurig               lustig
             37                    wunderbar            großartig
             38                     schuldig             beschämt
             39                        schön            wunderbar
             40               zuversichtlich               sicher
             41                         blöd       unterbelichtet
             42                  umfangreich                 groß
             43                         nett              grausam
             44                   ungeduldig               nervös
             45                         groß                breit
             46                        stark                stolz
             47                      unnötig            notwendig
             48                      rastlos                 jung
             49                         blöd          intelligent
             50                     schlecht            großartig
             51                    schwierig              einfach
             52                    notwendig              wichtig
             53                     schlecht          schrecklich
             54                         irre                 froh
             55                      ehrlich             schuldig
             56                       leicht                 hart
             57                       leicht             flexibel
             58                       gewiss               sicher
             

-----

### correcting datasets / applying manual Wiki-lemmatization

In [55]:
dic = load('dewiki', 'dict')
dic.token2id['Wahlfach']

Loading dict from ../data/preprocessed/LDAmodel/noun/bow/dewiki_noun_bow.dict


91802

In [9]:
lemmap = load('dewiki', 'lemmap')
lemmap

fLoading {file}


token
+10                                                                                                                     +10
+φ                                                                                                                       +φ
0+0                                                                                                                     0+0
0,0022″                                                                                                             0,0022″
0,01–0,1                                                                                                           0,01–0,1
0,1)-Form                                                                                                         0,1)-Form
0,1-Megapixel-Kamera                                                                                   0,1-Megapixel-Kamera
0,1-Schritt                                                                                                   0,1-Schritten
0,

In [49]:
# simlex
missing = ['Agression', 'Blase', 'Büderlichkeit', 'Decke', 'Dunstschleier', 'Ecke', 'Frieden', 'Gefährte', 'Gegener', 'Gerassel', 'Haufen', 'Haushelferin', 'Herde', 'Kleider', 'Kohle', 'Menschen', 'Männer', 'Nachrichten', 'Reise', 'Samen', 'Schaden', 'Zähne']
# ws
missing = ['Augen', 'Drogen', 'Frieden', 'Kleider', 'Kohle', 'Medien', 'Menschen', 'Nachrichten', 'Ohren', 'Präzedensfall', 'Reise', 'Runde', 'Truppen', 'abheben', 'beispielhaft', 'besiegen', 'ficken', 'klug', 'kämpfen', 'live', 'palestinensisch', 'säugen', 'trinken', 'virtuos', 'vorangehend', 'vorausgehend', 'vorbildlich']
# gut
missing = ['Aktensperrfrist', 'Arbeitssuchender', 'Baumaschinenmeister', 'Beamte', 'Betrugshandlung', 'Böse', 'Büroequipment', 'Detailkonstrukteurinnen', 'Endlast', 'Fachbereichsvertreter', 'Flächeneinsparung', 'Frieden', 'Frontenbildung', 'Frühlingssonne', 'Garnerzeugung', 'Gastlandkontakte', 'Geschirrdurcheinander', 'Gleitkomma', 'Hirnsignal', 'Körpernorm', 'Langzeittherapieprogramm', 'Linkstatistik', 'Makake', 'Managerinnen', 'Metadatenwerkzeug', 'Migrantinnen', 'Monate', 'Neoautoritarismus', 'Neuronenaktivität', 'Organisationskompetenz', 'Personaldisposition', 'Portfolioanalyse', 'Premium-Hersteller', 'Probenehmer', 'Quartalsumfrage', 'Quelle', 'Reise', 'Reiseschutzpass', 'Restaurierungsmethode', 'Samenzahnrad', 'Sandwich-Konzept', 'Schaden', 'Soziales', 'Spitze', 'Sports-Tourer', 'Studierende', 'TV-Kamera', 'Tiefbaubauingenieur', 'Ursprugsort', 'Verkäuferinnen', 'Vertriebstechniker', 'Vliesstofferzeugung', 'Volierenzelt', 'Wahlfächer', 'Warte', 'Weiblich']
for word in missing:
    print(word, '->')
    lem = lemmap[lemmap == word].index.values
    print(lem)
    if len(lem) == 0:
        print(list(lemmap[lemmap.str.startswith(word[:-2])].index.values))
    try:
        print(lemmap[word])
    except KeyError as e:
        print('error', e)
        
    print()

Aktensperrfrist ->
[]
[]
error 'Aktensperrfrist'

Arbeitssuchender ->
[]
['Arbeitssuchende']
error 'Arbeitssuchender'

Baumaschinenmeister ->
[]
[]
error 'Baumaschinenmeister'

Beamte ->
[]
['Beam', 'Beam-Außenlaststation', 'Beam-Familie', 'Beam-Power-Tetrode', 'Beam-Scanning', 'Beam-Suntory-Konzern', 'Beam-Verlag', 'Beaman', 'Beaman-Gletscher', 'Beamaufnahme', 'Beambten', 'Beamdog', 'Beamdogs', 'Beame', 'Beamen', 'Beamer', 'Beamer-Leinwand', 'Beamer-Präsentation', 'Beamersystem', 'Beamfleot', 'Beamformer', 'Beamforming', 'Beaming', 'Beaminster', 'Beamish', 'Beamline', 'Beamlines', 'Beamoa', 'Beamon', 'Beamons', 'Beamont', 'Beamor', 'Beamptung', 'Beams', 'Beamshow', 'Beamsville', 'BeamtStG', 'BeamtStG.', 'BeamtVG', 'Beamte/Angestellte', 'Beamten-Ausbildung', 'Beamten-Bau-', 'Beamten-Dienstrechtsgesetz', 'Beamten-Elite', 'Beamten-Erholungsheim', 'Beamten-Hierarchie', 'Beamten-Krankenversicherung', 'Beamten-Laufbahn', 'Beamten-Selbsthilfewerk', 'Beamten-Verein', 'Beamten-Versicherung', '

[]
error 'Managerinnen'

Metadatenwerkzeug ->
[]
[]
error 'Metadatenwerkzeug'

Migrantinnen ->
['Migrantin']
error 'Migrantinnen'

Monate ->
['Monat']
error 'Monate'

Neoautoritarismus ->
[]
[]
error 'Neoautoritarismus'

Neuronenaktivität ->
['Neuronenaktivität']
Neuronenaktivität

Organisationskompetenz ->
['Organisationskompetenz']
Organisationskompetenz

Personaldisposition ->
['Personaldisposition']
Personaldisposition

Portfolioanalyse ->
['Portfolioanalyse']
Portfolioanalyse

Premium-Hersteller ->
['Premium-Hersteller']
Premium-Hersteller

Probenehmer ->
['Probenehmer']
Probenehmer

Quartalsumfrage ->
[]
[]
error 'Quartalsumfrage'

Quelle ->
[]
['Quel', 'Quela', 'Quelae', 'Quelaines', 'Quelaines-Saint-Gault', 'Quelan', 'Quelapa', 'Quelcata', 'Quelccaya-Eiskappe', 'Quelch', 'Quele', 'Quele-Boro-Uai', 'Quelea', 'Quelea"-Art', 'Quelen', 'Queler', 'Quelet', 'Quelet-Reaktion', 'Quelfes', 'Quelhas', 'Quelicai', 'Quelicais', 'Quelimane', 'Quelin', 'Queling', 'Quelite', 'Quelitz', 'Quelk

[]
['Restaurierungsmethode', 'Restaurierungsmethodik']
Restaurierungsmethoden

Samenzahnrad ->
[]
[]
error 'Samenzahnrad'

Sandwich-Konzept ->
['Sandwich-Konzept']
Sandwich-Konzept

Schaden ->
[]
['Schad', "Schad'n", 'Schad)Software', 'Schad-Ros', 'Schad-Rossas', 'Schadaeus', 'Schadan', 'Schadanfälligkeit', 'Schadans', 'Schadau', 'Schadaupark', 'Schadaus', 'Schadausmaß', 'Schadbach', 'Schadberg', 'Schadbestand', 'Schadbild', 'Schadburg', 'Schadbär', 'Schadchen', 'Schadcode', 'Schadda', 'Schaddach', 'Schaddad', 'Schaddadide', 'Schaddadiden', 'Schaddai', 'Schaddaj', 'Schaddel', 'Schaddelmühle', 'Schaddingsdorf', 'Schaddisposition', 'Schaddād', 'Schade-Ahausen', 'Schade-Lindig', 'Schade-Salwey', 'Schadeberg', 'Schadeberg-Herrmann', 'Schadebeuster', 'Schadebrodt', 'Schadeburg', 'Schadeburgstraße', 'Schadeck', 'Schadee', 'Schadeffekt', 'Schadefähre', 'Schadegan', 'Schadegard', 'Schadegards', 'Schadegeld', 'Schadegg', 'Schadegur', 'Schadehop', 'Schadehorn', 'Schadeinheit', 'Schadeinwirkung',

[]
[]
error 'Sports-Tourer'

Studierende ->
[]
['Studieren', 'Studierende(r', 'Studierenden-Nationalmannschaft', 'Studierenden-Parlament', 'Studierenden-Service-Center', 'Studierenden-Sozialerhebung', 'Studierenden-Team', 'Studierenden-Weltmeisterschaft', 'Studierendenakademie', 'Studierendenangelegenheit', 'Studierendenanzahl', 'Studierendenapartment', 'Studierendenausschuss', 'Studierendenaustausch', 'Studierendenauswahl', 'Studierendenausweis', 'Studierendenbeitrag', 'Studierendenberatung', 'Studierendenbereich', 'Studierendenbetreuung', 'Studierendenbewegung', 'Studierendenchapters', 'Studierendendichte', 'Studierendenexekutive', 'Studierendenfachbereich', 'Studierendenfilmfestival', 'Studierendenforum', 'Studierendenförderung', 'Studierendengemein', 'Studierendengemeinde', 'Studierendengemeinschaft', 'Studierendengesellschaft', 'Studierendengruppe', 'Studierendenhaus', 'Studierendenheim', 'Studierendeninitiative', 'Studierendenjahrgang', 'Studierendenkongress', 'Studierendenmobili